# 1. Import necessary libraries for data handling and visualization.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
#import ee
#import geemap

# 2. The eSBAE function:

In [2]:
def calculate_areas(db_total, strata_column, categories_column, total_area, z_score):
    
    df_full = db_total.copy()
    df_interpreted = db_total[~db_total[categories_column].isna()]
    
    # get all attributes
    categories = df_interpreted[categories_column].unique()
    
    # get strata
    strata, d = df_interpreted[strata_column].unique(), {}
    print(categories)
    # create stats for each entry
    for category in categories:
        
        if str(category) == 'nan':
            continue
            
        print(f' Calculating stats for {category}')
        # create binary class column
        df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
        print(f'There are {df_interpreted[category].sum()} entries of {category} in {categories_column}.')
        
        # initialize variables for category <> check the catergories > def, deg, gain
        categories_area, se_total = 0, 0
        d2 = {}
        for stratum in strata:
                        
            if str(stratum) == 'nan':
                continue
            
            # subset to stratum
            df_stratum = df_full[df_full[strata_column] == stratum]
            
            # get area proportion for that stratum on full dataset
            proportion_strata = len(df_stratum)/len(df_full)

            # get stratum area
            stratum_area = proportion_strata * total_area

            # get proportion of forest change within strata from interpreted data
            proportion_category = len(
                df_interpreted[(df_interpreted[strata_column] == stratum) & (df_interpreted[category] == 1)]
            ) / len(
                df_interpreted[df_interpreted[strata_column] == stratum]
            )
            
            # get area from proportion and full area
            category_stratum_area = proportion_category * stratum_area

            # get error from interpreted data for full stratum area
            var = np.var(df_interpreted[category][df_interpreted[strata_column] == stratum])
            sd = np.sqrt(var)               
            n = len(df_interpreted[df_interpreted[strata_column] == stratum])
            se = sd/np.sqrt(n) * stratum_area
                          
            # add for totals
            categories_area += category_stratum_area
            se_total += se**2
            
            # add to dictionary
            d2[f'area_stratum_{stratum}'] = category_stratum_area
            d2[f'ci_stratum_{stratum}']=z_score*se
        
        d2['area_total'] = categories_area
        d2['MOE'] = z_score*np.sqrt(se_total)
        d2['MOE_perc'] =  d2['MOE'] / categories_area * 100
        d[category] = d2
    
    
    return pd.DataFrame.from_dict(d, orient='index')

# 3 Load and prepare interpreted data from CEO Validation 
#### COG: 1001 points [07/12/2023]
#### set 1 = 1001 pts + set 2 = 2000 pts [13/12/2023]

In [3]:
#CEO_pts = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set1-sample-data-2023-12-06.csv', delimiter=';')
CEO_set1 = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set1-sample-data-2023-12-13.csv', delimiter=',')
CEO_set2 = pd.read_csv('/home/sepal-user/eSBAE_COG/data/ceo-NERF_2016_2022_CONGO_set2-sample-data-2023-12-13.csv', delimiter=',')

In [ ]:
len(CEO_set1) # = 1001

In [ ]:
len(CEO_set2) # = 2000

In [ ]:
CEO_set1.columns.to_list() 
#CEO_pts.head()

## 3.1 Different data checks and harmonisations 

In [4]:
# vérification doublons set 1
duplicated = CEO_set1['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

problem


In [5]:
doublonsCEO_set1 = CEO_set1[CEO_set1.duplicated(subset='plotid', keep=False)]
print(doublonsCEO_set1)

   plotid  sampleid        lon       lat                      email  flagged  \
7  541957    541957  11.235013 -3.836062      menguekarel@gmail.com    False   
8  541957    541957  11.235013 -3.836062  ndandoularissa7@gmail.com    False   

    collection_time analysis_duration imagery_title  imagery_attributions  \
7  2023-11-23 08:44        132.6 secs           NaN                   NaN   
8  2023-11-23 09:08        331.0 secs           NaN                   NaN   

   ... Type du changement 2 (1)  Type de moteur pour changement 2 (1)  \
7  ...                      NaN                                   NaN   
8  ...                      NaN                                   NaN   

   Le feu a t-il causé le changement ?.1  Décrivez autres (1) (0)  \
7                                    NaN                      NaN   
8                                    NaN                      NaN   

   Y-a t-il eu régénération  ?  Indiquez la date de la régénération  \
7                          N

In [6]:
CEO_set1['doublon'] = 'no'
CEO_set1['doublon'] = np.where((CEO_set1['plotid'] == 541957) & (CEO_set1['Commentaires'] == 'rentrez vos commentaires'), 'yes', CEO_set1['doublon'])

count_values_db = CEO_set1['doublon'].value_counts()
print(count_values_db)

doublon
no     1000
yes       1
Name: count, dtype: int64


In [7]:
dfCEO_set1 = CEO_set1[CEO_set1['doublon'] == 'no']
len(dfCEO_set1)

1000

In [8]:
# vérification doublons set 2
duplicated = CEO_set2['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

all good


In [9]:
CEO_set1['source'] = 'set1'
CEO_set2['source'] = 'set2'

In [10]:
CEO_cols = [ 'plotid',
 'forêt ou non-forêt en 2016?',
 'Type de non-forêt en 2016',
 'Type de forêt en 2016',
 'Y-a t-il un changement négatif sur la période 2016-2022',
 'Quel type de changement ? ',
 "Indiquez l'année du changement 1",
 'Type de moteur pour changement 1',
 'Le feu a t-il causé le changement ?',
 'Décrivez autres',
 'y-a t-il un second changement ?',
 'Type du changement 2 (1)',
 'Type de moteur pour changement 2 (1)',
 'Le feu a t-il causé le changement ?.1',
 'Décrivez autres (1) (0)',
 'Y-a t-il eu régénération  ?',
 'Indiquez la date de la régénération',
 'Définir la strate en 2022 ',
 'Type de non-forêt en 2022',
 'Type de forêt en 2022',
 'Commentaires',
  'source']

In [11]:
# concatenation set 1 + set 2
dfCEOconcat = pd.concat([CEO_set1[CEO_cols], CEO_set2[CEO_cols]], axis=0, ignore_index=True)
len(dfCEOconcat) # = 3001

3001

In [12]:
# vérification doublons set 2
doublon = dfCEOconcat['plotid'].duplicated().any() 
if duplicated:
    print ("problem")
else:
    print ("all good")

all good


In [ ]:
#activites = dfCEO_pts_clean['Activités '].value_counts()
#print(activites)

In [ ]:
pivot1 = pd.pivot_table(dfCEOconcat,values='source',index=['forêt ou non-forêt en 2016?'],columns=['Type de non-forêt en 2016'],aggfunc="count")
pivot1

In [ ]:
pivot2 = pd.pivot_table(dfCEOconcat,values='source',index=['forêt ou non-forêt en 2016?'],columns=['Type de forêt en 2016'],aggfunc="count")
pivot2

In [13]:
## définition des catégories au niveau 1 (IPCC) - 2016
dfCEOconcat['ipcc_lulc_2016'] = 'problem'
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['forêt ou non-forêt en 2016?'] == 'forêt'), 'Terres forestieres', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == ' zone baties'), 'Etablissement humain', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'eau '), 'Terres humides', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'je ne sais pas'), 'Autres terres', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'prairie aquatique'), 'Terres humides', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'savane arborée/arbustive'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'savane herbacée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'sol nu végétation éparse'), 'Autres terres', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'terres cultivées annuelles'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['Type de non-forêt en 2016'] == 'terres cultivées permanentes'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2016'])
dfCEOconcat['ipcc_lulc_2016'] = np.where((dfCEOconcat['forêt ou non-forêt en 2016?'] == 'NaN'), 'invalide', dfCEOconcat['ipcc_lulc_2016']) ### ne marche pas ...???

count1 = dfCEOconcat['ipcc_lulc_2016'].value_counts()
print(count1)

ipcc_lulc_2016
Terres forestieres      1476
Terres gramineennes      987
problem                  270
Terres cultivees         126
Terres humides           101
Autres terres             30
Etablissement humain      11
Name: count, dtype: int64


In [ ]:
pivot3 = pd.pivot_table(dfCEOconcat,values='plotid',index=['forêt ou non-forêt en 2016?'],columns=['ipcc_lulc_2016'],aggfunc="count")
pivot3

In [14]:
## définition des catégories au niveau 2 (classes nationales) - 2016
dfCEOconcat['n2_lulc_2016'] = dfCEOconcat['Type de non-forêt en 2016']
dfCEOconcat.loc[dfCEOconcat['forêt ou non-forêt en 2016?'] == 'forêt', 'n2_lulc_2016'] = dfCEOconcat['Type de forêt en 2016']

In [15]:
pivot3b = pd.pivot_table(dfCEOconcat,values='plotid',index=['n2_lulc_2016'],columns=['ipcc_lulc_2016'],aggfunc="count")
pivot3b

ipcc_lulc_2016,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
n2_lulc_2016,,,,,,
zone baties,NaN,11.0,NaN,NaN,NaN,NaN
1 - forêt dense,NaN,NaN,NaN,694.0,NaN,NaN
10 - plantation forestière,NaN,NaN,NaN,7.0,NaN,NaN
3 - forêt secondaire,NaN,NaN,NaN,522.0,NaN,NaN
4- forêt claire,NaN,NaN,NaN,20.0,NaN,NaN
7 - forêt mangrove,NaN,NaN,NaN,1.0,NaN,NaN
8 - forêt marécageuse,NaN,NaN,NaN,115.0,NaN,NaN
9 - forêt galérie,NaN,NaN,NaN,117.0,NaN,NaN
eau,NaN,NaN,NaN,NaN,NaN,53.0


In [16]:
pivot4 = pd.pivot_table(dfCEOconcat,values='plotid',index=['Définir la strate en 2022 '],columns=['Type de non-forêt en 2022'],aggfunc="count")
pivot4

Type de non-forêt en 2022,eau,je ne sais pas,prairie aquatique,savane arbustive/arborée,savane herbacée,sol nu végétation éparse,terres cultivées annuelles,terres cultivées permanentes,zone baties
Définir la strate en 2022,,,,,,,,,
non-forêt,54,1,48,408,580,47,130,16,11


In [17]:
pivot5 = pd.pivot_table(dfCEOconcat,values='plotid',index=['Définir la strate en 2022 '],columns=['Type de forêt en 2022'],aggfunc="count")
pivot5

Type de forêt en 2022,1 - forêt dense,10 - plantation forestière,3 - forêt secondaire,4- forêt claire,8 - forêt marécageuse,9 - forêt galérie
Définir la strate en 2022,,,,,,
forêt,661,6,519,20,115,115


In [18]:
## définition des catégories au niveau 1 (IPCC) - 2022
dfCEOconcat['ipcc_lulc_2022'] = 'problem'
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Définir la strate en 2022 '] == 'forêt'), 'Terres forestieres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == ' zone baties'), 'Etablissement humain', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'eau '), 'Terres humides', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'je ne sais pas'), 'Autres terres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'prairie aquatique'), 'Terres humides', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'savane arbustive/arborée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'savane herbacée'), 'Terres gramineennes', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'sol nu végétation éparse'), 'Autres terres', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'terres cultivées annuelles'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2022'])
dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Type de non-forêt en 2022'] == 'terres cultivées permanentes'), 'Terres cultivees', dfCEOconcat['ipcc_lulc_2022'])
#dfCEOconcat['ipcc_lulc_2022'] = np.where((dfCEOconcat['Définir la strate en 2022 '] == 'NaN'), 'invalide', dfCEOconcat['ipcc_lulc_2022']) ### ne marche pas ...???

count1 = dfCEOconcat['ipcc_lulc_2022'].value_counts()
print(count1)

ipcc_lulc_2022
Terres forestieres     1436
Terres gramineennes     988
problem                 335
Terres cultivees        146
Autres terres            48
Terres humides           48
Name: count, dtype: int64


In [19]:
## définition des catégories au niveau 2 (classes nationales) - 2022
dfCEOconcat['n2_lulc_2022'] = dfCEOconcat['Type de non-forêt en 2022']
dfCEOconcat.loc[dfCEOconcat['Définir la strate en 2022 '] == 'forêt', 'n2_lulc_2022'] = dfCEOconcat['Type de forêt en 2022']

In [20]:
pivot5b = pd.pivot_table(dfCEOconcat,values='plotid',index=['n2_lulc_2022'],columns=['ipcc_lulc_2022'],aggfunc="count")
pivot5b

ipcc_lulc_2022,Autres terres,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides,problem
n2_lulc_2022,,,,,,
1 - forêt dense,NaN,NaN,661.0,NaN,NaN,NaN
10 - plantation forestière,NaN,NaN,6.0,NaN,NaN,NaN
3 - forêt secondaire,NaN,NaN,519.0,NaN,NaN,NaN
4- forêt claire,NaN,NaN,20.0,NaN,NaN,NaN
8 - forêt marécageuse,NaN,NaN,115.0,NaN,NaN,NaN
9 - forêt galérie,NaN,NaN,115.0,NaN,NaN,NaN
eau,NaN,NaN,NaN,NaN,NaN,54.0
je ne sais pas,1.0,NaN,NaN,NaN,NaN,NaN
prairie aquatique,NaN,NaN,NaN,NaN,48.0,NaN


In [21]:
#matrice des transitions n1 ipcc
tmatrix_n1 = pd.pivot_table(dfCEOconcat,values='plotid',index=['ipcc_lulc_2016'],columns=['ipcc_lulc_2022'],aggfunc="count")
tmatrix_n1

ipcc_lulc_2022,Autres terres,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides,problem
ipcc_lulc_2016,,,,,,
Autres terres,27.0,NaN,2.0,1.0,NaN,NaN
Etablissement humain,NaN,NaN,NaN,NaN,NaN,11.0
Terres cultivees,NaN,120.0,5.0,1.0,NaN,NaN
Terres forestieres,19.0,19.0,1424.0,14.0,NaN,NaN
Terres gramineennes,2.0,7.0,5.0,972.0,NaN,1.0
Terres humides,NaN,NaN,NaN,NaN,48.0,53.0
problem,NaN,NaN,NaN,NaN,NaN,270.0


In [22]:
tmatrix_n2 = pd.pivot_table(dfCEOconcat,values='plotid',index=['ipcc_lulc_2016', 'n2_lulc_2016'],columns=['ipcc_lulc_2022', 'n2_lulc_2022'],aggfunc="count")
tmatrix_n2

ipcc_lulc_2022                                     Autres terres  \
n2_lulc_2022                                      je ne sais pas   
ipcc_lulc_2016       n2_lulc_2016                                  
Autres terres        je ne sais pas                          1.0   
                     sol nu végétation éparse                NaN   
Etablissement humain  zone baties                            NaN   
Terres cultivees     terres cultivées annuelles              NaN   
                     terres cultivées permanentes            NaN   
Terres forestieres   1 - forêt dense                         NaN   
                     10 - plantation forestière              NaN   
                     3 - forêt secondaire                    NaN   
                     4- forêt claire                         NaN   
                     7 - forêt mangrove                      NaN   
                     8 - forêt marécageuse                   NaN   
                     9 - forêt galérie                       NaN   
Terres gramineennes  savane arborée/arbustive                NaN   
                     savane herbacée                         NaN   
Terres humides       eau                                     NaN   
                     prairie aquatique                       NaN   

ipcc_lulc_2022                                                              \
n2_lulc_2022                                      sol nu végétation éparse   
ipcc_lulc_2016       n2_lulc_2016                                            
Autres terres        je ne sais pas                                    NaN   
                     sol nu végétation éparse                         26.0   
Etablissement humain  zone baties                                      NaN   
Terres cultivees     terres cultivées annuelles                        NaN   
                     terres cultivées permanentes                      NaN   
Terres forestieres   1 - forêt dense                                   5.0   
                     10 - plantation forestière                        1.0   
                     3 - forêt secondaire                             12.0   
                     4- forêt claire                                   NaN   
                     7 - forêt mangrove                                NaN   
                     8 - forêt marécageuse                             NaN   
                     9 - forêt galérie                                 1.0   
Terres gramineennes  savane arborée/arbustive                          1.0   
                     savane herbacée                                   1.0   
Terres humides       eau                                               NaN   
                     prairie aquatique                                 NaN   

ipcc_lulc_2022                                              Terres cultivees  \
n2_lulc_2022                                      terres cultivées annuelles   
ipcc_lulc_2016       n2_lulc_2016                                              
Autres terres        je ne sais pas                                      NaN   
                     sol nu végétation éparse                            NaN   
Etablissement humain  zone baties                                        NaN   
Terres cultivees     terres cultivées annuelles                        106.0   
                     terres cultivées permanentes                        NaN   
Terres forestieres   1 - forêt dense                                     1.0   
                     10 - plantation forestière                          NaN   
                     3 - forêt secondaire                               17.0   
                     4- forêt claire                                     NaN   
                     7 - forêt mangrove                                  NaN   
                     8 - forêt marécageuse                               NaN   
                     9 - forêt galérie                                   NaN   
Terres gramineennes  sav

In [25]:
## définition des activités
dfCEOconcat['DA1622'] = 'problem'
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] == 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] != 'Terres forestieres'), 'Def', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] != 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] == 'Terres forestieres'), 'Gain', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] == 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] == 'Terres forestieres'), 'SF', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['ipcc_lulc_2016'] != 'Terres forestieres') & (dfCEOconcat['ipcc_lulc_2022'] != 'Terres forestieres'), 'SNF', dfCEOconcat['DA1622'])

#degradation
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '1 - forêt dense') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '8 - forêt marécageuse') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])
dfCEOconcat['DA1622'] = np.where((dfCEOconcat['n2_lulc_2016'] == '9 - forêt galérie') & (dfCEOconcat['n2_lulc_2022'] == '3 - forêt secondaire'), 'Deg', dfCEOconcat['DA1622'])

count1 = dfCEOconcat['DA1622'].value_counts()
print(count1)

DA1622
SNF     1513
SF      1394
Def       52
Deg       30
Gain      12
Name: count, dtype: int64


#### 4. load national grid

In [27]:
#ee.Initialize()
#df_COG_grid = ee.FeatureCollection("users/andreasvollrath/COGSamples")

## scp -P 443 -r arquero@ssh.sepal.io:/home/sepal-user/module_results/esbae/my_first_esbae_project_congo_v1/COG_esbae_2015_2022_model.csv/ . (demander mdp à Amélie)
df_COG_grid = pd.read_csv('/home/sepal-user/eSBAE_COG/data/COG_esbae_2015_2022_model.csv', delimiter=',')
#len(df_COG_grid) # = 291595

In [ ]:
# List all columns 
df_COG_grid.columns.tolist()

## 5. FULL dataframe with national GRID + 1k interpreted points

In [28]:
df_COG_grid.dtypes

images                 int64
mon_images             int64
bfast_change_date    float64
bfast_magnitude      float64
bfast_means          float64
                      ...   
stratum                int64
kmeans                 int64
PLOTID                 int64
LON                  float64
LAT                  float64
Length: 87, dtype: object

In [29]:
dfCEOconcat.dtypes

plotid                                                       int64
forêt ou non-forêt en 2016?                                 object
Type de non-forêt en 2016                                   object
Type de forêt en 2016                                       object
Y-a t-il un changement négatif sur la période 2016-2022     object
Quel type de changement ?                                   object
Indiquez l'année du changement 1                           float64
Type de moteur pour changement 1                            object
Le feu a t-il causé le changement ?                         object
Décrivez autres                                             object
y-a t-il un second changement ?                             object
Type du changement 2 (1)                                    object
Type de moteur pour changement 2 (1)                        object
Le feu a t-il causé le changement ?.1                       object
Décrivez autres (1) (0)                                     ob

In [30]:
dfCEOconcat.rename(columns={'plotid':'PLOTID'}, inplace=True)

In [31]:
df_COG_esbae = df_COG_grid[['PLOTID', 'kmeans']].merge(dfCEOconcat[['PLOTID', 'DA1622']], how='left', on='PLOTID')
len(df_COG_esbae)

291596

In [32]:
activites2 = df_COG_esbae['DA1622'].value_counts()
print(activites2)

DA1622
SNF     1513
SF      1394
Def       52
Deg       30
Gain      12
Name: count, dtype: int64


##### Perform area calculation using the stratum column. In this case the column is called kmeans. Use the merge dataframe (national grid points + CEO validated points)

## 6. Run the eSBAE function for the different scenarios

In [33]:
calculate_areas(db_total=df_COG_esbae, strata_column='kmeans', categories_column='DA1622', total_area=len(df_COG_esbae), z_score=1.645)

['SNF' 'SF' 'Deg' 'Gain' 'Def']
 Calculating stats for SNF
There are 1513 entries of SNF in DA1622.
 Calculating stats for SF
There are 1394 entries of SF in DA1622.
 Calculating stats for Deg
There are 30 entries of Deg in DA1622.
 Calculating stats for Gain
There are 12 entries of Gain in DA1622.
 Calculating stats for Def
There are 52 entries of Def in DA1622.


/tmp/ipykernel_432/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_432/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_432/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_stratum_3,ci_stratum_3,area_total,MOE,MOE_perc
SNF,41448.238606,1321.724739,43327.170172,6038.847549,9478.576065,545.269449,94253.984843,6205.800092,6.584125
SF,22805.202413,1316.167350,162673.829828,6038.847549,9648.215010,546.029341,195127.247251,6204.685670,3.179815
Deg,303.491287,188.253008,0.000000,0.000000,487.711968,165.325591,791.203254,250.542903,31.666061
Gain,43.355898,71.296547,0.000000,0.000000,233.253550,115.043390,276.609448,135.344669,48.929879
Def,86.711796,100.794726,0.000000,0.000000,1060.243408,240.317805,1146.955204,260.599739,22.721004
